In [3]:
# 
# labels[0] => Week #
# labels[1] => Day of Week
# labels[2] => Backup Start Time - Hour of Day
# labels[3] => Work-Flow-ID
# labels[4] => File Name
# labels[5] => Size of Backup (GB)
# labels[6] => Backup Time (hour)
# 
import numpy as np
import pandas as pd

Data = pd.read_csv("../data/network_backup_dataset.csv")
labels = Data.columns
for i, v in enumerate(labels):
    print "labels[" + str(i) + "] => " + v
    
uniqueFiles = ['File_{0}'.format(s) for s in xrange(len((pd.unique(Data['File Name']))))]

def OneDimensionTransfer(label, data):
    if label == labels[1]:
        return data.replace({'Day of Week': {'Monday' : 0, 'Tuesday' : 1, 'Wednesday' : 2, 'Thursday' : 3, 'Friday' : 4,'Saturday' : 5, 'Sunday' : 6}})
    elif label == labels[3]:
        return data.replace({'Work-Flow-ID': {'work_flow_0' : 0, 'work_flow_1' : 1, 'work_flow_2' : 2, 'work_flow_3' : 3, 'work_flow_4' : 4}})
    elif label == labels[4]:
        for i,j in zip(uniqueFiles,range(len(uniqueFiles))):
            data = data.replace({'File Name': {i : j}})
        return data
    else:
        return data
    
# for i in range(len(labels)):    
#     Data = OneDimensionTransfer(labels[i], Data)
    
# newData = Data

def OneHotEncoding(label, data):
    tmpData = pd.get_dummies(data,columns=[str(label)])
    return tmpData

# newData = OneHotEncoding(labels[1], newData)
# print newData

labels[0] => Week #
labels[1] => Day of Week
labels[2] => Backup Start Time - Hour of Day
labels[3] => Work-Flow-ID
labels[4] => File Name
labels[5] => Size of Backup (GB)
labels[6] => Backup Time (hour)


In [4]:
import itertools
def MakeCombinationData(data):
    lsts = list(itertools.product([0, 1], repeat=5))
    data_set = []
    for lst in lsts:
        tmp_data = data
        for index,item in enumerate(lst):
            if item == 0:
                tmp_data = OneDimensionTransfer(labels[index], tmp_data)
            else:
                tmp_data = OneHotEncoding(labels[index], tmp_data)
        data_set.append(tmp_data)
    return data_set    
Data_set = MakeCombinationData(Data)
print len(Data_set)

32


In [39]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import cross_val_predict
from sklearn.model_selection import KFold


Data = pd.read_csv("../data/network_backup_dataset.csv")
labels = Data.columns

uniqueFiles = ['File_{0}'.format(s) for s in xrange(len((pd.unique(Data['File Name']))))]

def OneDimensionTransfer(label, data):
    if label == labels[1]:
        return data.replace({'Day of Week': {'Monday' : 0, 'Tuesday' : 1, 'Wednesday' : 2, 'Thursday' : 3, 'Friday' : 4,'Saturday' : 5, 'Sunday' : 6}})
    elif label == labels[3]:
        return data.replace({'Work-Flow-ID': {'work_flow_0' : 0, 'work_flow_1' : 1, 'work_flow_2' : 2, 'work_flow_3' : 3, 'work_flow_4' : 4}})
    elif label == labels[4]:
        for i,j in zip(uniqueFiles,range(len(uniqueFiles))):
            data = data.replace({'File Name': {i : j}})
        return data
    else:
        return data
    
for i in range(len(labels)):    
    Data = OneDimensionTransfer(labels[i], Data)
    
Training = Data.copy()
Training.drop(labels[6],1,inplace = True)
#Standard = Training.pop(labels[5])

Lin_reg = LinearRegression(fit_intercept = False)


def CalculateRMSE(data,predict):
    return sp.sqrt(sp.mean((predict - data) ** 2))
def FoldRMSE(data):
    target = data.pop(labels[5])
    Lin_reg.fit(data,target)
    predict = Lin_reg.predict(data)
    return CalculateRMSE(predict,target)
kf = KFold(n_splits=10)
kf.get_n_splits(Training)
for train_index, test_index in kf.split(Training):
    train_set = Training.iloc[train_index, :]
    test_set = Training.iloc[test_index, :]
    print 'train_RMSE', FoldRMSE(train_set)
    print 'test_RMSE', FoldRMSE(test_set)

train_RMSE 0.10393858396050151
test_RMSE 0.10706967581259685
train_RMSE 0.10475965294901611
test_RMSE 0.09993815074384507
train_RMSE 0.1039705690187425
test_RMSE 0.10671561624213878
train_RMSE 0.10479786936473291
test_RMSE 0.10003201367711427
train_RMSE 0.10397163515851572
test_RMSE 0.10686486436465822
train_RMSE 0.10481515905452862
test_RMSE 0.10005178076551445
train_RMSE 0.10399598739966495
test_RMSE 0.10677062820283881
train_RMSE 0.10481364331307322
test_RMSE 0.10003310552625817
train_RMSE 0.10397812819039667
test_RMSE 0.10670807152713645
train_RMSE 0.10483669246208886
test_RMSE 0.09954456918896258
